In [ ]:
# import packages here to reduce the size of code cells later

import pandas as pd
from prettypandas import PrettyPandas
import patsy

import numpy as np
import scipy.stats
import statsmodels.formula.api
import statsmodels.api as sm

from graphviz import Digraph
import seaborn as sns

import dexpy.factorial
import dexpy.alias
import dexpy.power

from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import patches

from IPython.display import display, Markdown, HTML

import warnings
#warnings.filterwarnings('ignore')

In [ ]:
# CSV Results Layout and File

columns = ['trial', 'lh', 'ps', 'id', 'rw', 'wt', 'cost', 'time', 'quality', 'comment']

cr6_print_history = pd.read_csv('https://raw.githubusercontent.com/wilsongis/3DP_Experiments/main/Data/cr6-doe-schedule.csv', skiprows=1, names=columns)

In [ ]:
# helper functions for this notebook

# use SVG for matplotlib-based figures
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

def coded_to_actual(coded_data, actual_lows, actual_highs):
    """Converts a pandas DataFrame from coded units to actuals."""
    actual_data = coded_data.copy()
    for col in actual_data.columns:
        if not (col in actual_highs and col in actual_lows):
            continue
        try:
            # convert continuous variables to their actual value
            actual_data[col] *= 0.5 * (float(actual_highs[col]) - float(actual_lows[col]))
            # don't need to cast to float here, if either are not a float exception will have been thrown
            actual_data[col] += 0.5 * (actual_highs[col] + actual_lows[col])
        except ValueError:
            # assume 2 level categorical
            actual_data[col] = actual_data[col].map({-1: actual_lows[col], 1: actual_highs[col]})
    return actual_data
        
def get_tick_labels(key, lows, highs, units):
    """Returns a list of low/high labels with units (e.g. [8mm, 10mm])"""
    return [str(lows[key]) + units[key], str(highs[key]) + units[key]]

# Motivating Example: Better Office Coffee

 * 5 input factors
  * lh of Coffee (2.5 to 4.0 oz.)
  * Grind size (8-10mm)
  * Brew time (3.5 to 4.5 minutes)
  * Grind Type (burr vs blade)
  * Coffee wt (light vs dark)
 * 1 response: Average overall liking by a panel of 5 office coffee addicts
  * Each taster rates the coffee from 1-9
 * Maximum of 3 taste tests a day, for liability reasons
 

In [ ]:
# Layer Thickness = lh
lh_low = .16 
lh_hi = .24 

# Print Speed = ps
ps_low = 50 
ps_hi = 60 

# Infill Density = id
id_low = .25 
id_hi = .15 

# Raster Width = rw
rw_low = .4 
rw_hi = .8 

# Wall Thicknessv = wt
wt_low = 1.2 
wt_hi = .8

In [ ]:
# set some variables related to the coffee data set
actual_lows = { 'lh' : lh_low, 'ps' : ps_low, 'id': id_low, 'rw': rw_low, 'wt': wt_low }
actual_highs = { 'lh' : lh_hi, 'ps' : ps_hi, 'id': id_hi, 'rw': rw_hi, 'wt': wt_hi }
units = { 'lh' : 'mm', 'ps' : 'mm/s', 'id': '%', 'rw': 'mm', 'wt': 'mm' }

# Fractional Factorials

* Coffee experiment is 2<sup>5</sup> runs (32)
* We want to add 4 center point runs to check for curvature
* Total runs = 36, 3 per day if all testers are in the office
* Estimate experiment will take a month

# Fractional Factorials
* Power for the experiment is > 99%
* Full factorial is overkill
* Instead run 2<sup>5-1</sup> experiments, a "half fraction"


# Fractional Factorials in dexpy

https://statease.github.io/dexpy/design-build.html#module-dexpy.factorial

In [ ]:
help(dexpy.factorial.build_factorial)

In [ ]:


# cr6_print_history = dexpy.factorial.build_factorial(5, 2**(5-1))
# cr6_print_history.columns = ['lh', 'ps', 'id', 'rw', 'wt']
center_points = [
]

#columns = ['trial', 'lh', 'ps', 'id', 'rw', 'wt', 'cost', 'time', 'quality', 'comment']

#cr6_print_history = pd.read_csv('file:///Users/wilsonm/Dropbox%20(APSU%20GIS)/Projects%20(DB)/Dissertation/3DP_Experiments/Data/cr6-doe-schedule.csv', skiprows=1, names=columns)

#cr6_print_history = cr6_print_history.append(pd.DataFrame(center_points * 2, columns=cr6_print_history.columns))
cr6_print_history.index = np.arange(0, len(cr6_print_history))

display(Markdown("## 2<sup>(5-1)</sup> Factorial Design"))
display(PrettyPandas(cr6_print_history))

actual_design = coded_to_actual(cr6_print_history, actual_lows, actual_highs)
cr6_print_actual = actual_design
display(Markdown("Actual ## 2<sup>(5-1)</sup> Factorial Design"))
display(PrettyPandas(actual_design))

In [ ]:
# Transform DOE Schedule CSV dataframe to only include parameters

myDoE = cr6_print_history[['lh', 'ps', 'id', 'rw', 'wt']]
myDoE2 = cr6_print_actual[['lh', 'ps', 'id', 'rw', 'wt']]

### Cost Calculating Aliases in dexpy

In [ ]:
cr6_alias = cr6_print_history[['lh', 'ps', 'id', 'rw', 'wt']]

# Analysis

* [statsmodels](http://statsmodels.sourceforge.net/) has lots of routines for modeling data
* We will use [Ordinary Least Squares (OLS)](http://statsmodels.sourceforge.net/devel/examples/notebooks/generated/ols.html) to fit
* `statsmodels` typically takes `numpy` arrays for X and y data
* It also has a "formulas" api that accepts a `patsy` formula

## Cost Analysis

In [ ]:
cr6_cost_analysis = cr6_print_history[['lh', 'ps', 'id', 'rw', 'wt', 'cost']]
cr6_cost_analysis2 = cr6_print_actual[['lh', 'ps', 'id', 'rw', 'wt', 'cost']]
display(cr6_cost_analysis2)

In [ ]:
y_cost = cr6_cost_analysis['cost']
X = cr6_cost_analysis[['lh', 'ps', 'id', 'rw', 'wt']]

## An intercept is not added by default, so we need to add that here
X = sm.add_constant(X)
cost_results = sm.OLS(y_cost, X).fit()
cost_results.summary()

print(cost_results.summary())

In [ ]:
PlotWidth = 6
plt.figure(figsize=(PlotWidth, PlotWidth))
sns.regplot(x=cost_results.predict(X), y=y_cost)
plt.xlabel('Predicted %Conversion')
plt.ylabel('Actual %Conversion')
plt.title('Actual vs. Predicted %Conversion')
plt.show()

In [ ]:
display(cr6_cost_analysis)

In [ ]:
# Working Models
# quantreg
# poisson
# 

lm = statsmodels.formula.api.ols("cost ~(lh + ps + id + rw + wt)**2", data=cr6_cost_analysis).fit()
print(lm.summary())



In [ ]:
reduced_model = "rw"
lm = statsmodels.formula.api.ols("cost ~" + reduced_model, data=cr6_cost_analysis).fit()
print(lm.summary2())

# Visualization

* [seaborn](http://seaborn.pydata.org/) is built on top of `matplotlib` and adds support for `pandas` dataframes 
* Can build a plot using `seaborn`, then manipulate it with `matplotlib`
* Default themes look a lot nicer than what you get from `matplotlib` out of the box

In [ ]:
display(Markdown('''
If we take the experiment data from the design and use our new model to fit that data, then plot it against
the observed values we can get an idea for how well our model predicts. Points above the 45 degree line are
overpredicting for that combination of inputs. Points below the line predict a lower taste rating than
we actually measured during the experiment.'''))

actual_predicted = pd.DataFrame({ 'actual': cr6_print_history['cost'],
                                  'predicted': lm.fittedvalues
                                }, index=np.arange(len(cr6_cost_analysis['cost'])))
fg = sns.FacetGrid(actual_predicted, height=5)
fg.map(plt.scatter, 'actual', 'predicted')
ax = fg.axes[0, 0]
ax.plot([0, 1], [0, 1], color='g', lw=2)
ax.set_xticks(np.arange(0, 1.5))
ax.set_xlim([0, 1.])
ax.set_yticks(np.arange(0, 1))
ax.set_title('Actual vs Predicted')
_ = ax.set_ylim([0, 1])

In [ ]:
display(Markdown('''
Plotting the prediction for two factors at once shows how they interact with each other.
In this graph you can see that at the low brew time the larger grind size results in
a poor taste rating, likely because the coffee is too weak.'''))

pred_points = pd.DataFrame(1, columns = cr6_cost_analysis.columns, index=np.arange(4))
pred_points.loc[1,'wt'] = -1
pred_points.loc[3,'wt'] = -1
pred_points.loc[2,'rw'] = -1
pred_points.loc[3,'rw'] = -1
pred_points['cost'] = lm.predict(pred_points)
pred_points = coded_to_actual(pred_points, actual_lows, actual_highs)

fg = sns.factorplot('wt', 'cost', hue='rw', kind='point', data=pred_points)
ax = fg.axes[0, 0]
ax.set_xticklabels(get_tick_labels('wt', actual_lows, actual_highs, units))
_ = ax.set_title('Print Speed/Infill Density')

In [ ]:
display(Markdown('''
This graph contains the prediction with the highest taste rating, 7.72.
However, if you look at the dark bean line there is a point where we can get
a rating of 6.93 with 2.5oz of grounds.
'''))

pred_points = pd.DataFrame(1, columns = cr6_cost_analysis.columns, index=np.arange(4))
pred_points.loc[1,'lh'] = -1
pred_points.loc[3,'lh'] = -1
pred_points.loc[2,'wt'] = -1
pred_points.loc[3,'wt'] = -1
pred_points['cost'] = lm.predict(pred_points)
pred_points = coded_to_actual(pred_points, actual_lows, actual_highs)

fg = sns.factorplot('lh', 'cost', hue='wt', kind='point', palette={'dark': 'maroon', 'light': 'peru'}, data=pred_points)
ax = fg.axes[0, 0]
ax.set_xticklabels(get_tick_labels('lh', actual_lows, actual_highs, units))
ax.set_title('lh/wt Interaction')
plt.show()

display(PrettyPandas(pred_points))
display(Markdown('''That savings of 1.5oz per pot would create a nice surplus in the coffee budget at the end of the year.'''))

![coffeemaker](img/coffee_maker.jpg)

# The End

* We were able to build and execute an experiment in Python that resulted in a better tasting (and cheaper!) coffee.
* These slides can be found at https://hpanderson.github.io/dexpy-pymntos
* The jupyter notebook they are based on can be found on my github: https://github.com/hpanderson/dexpy-pymntos
* You can reach me at: <hank@statease.com>
* The `dexpy` docs are at: https://statease.github.io/dexpy/
* `dexpy` is only at version 0.1, we plan on greatly expanding the design and analysis capabilities
 